In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import acnportal
import sklearn
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import seaborn as sns

from copy import deepcopy
import warnings
import pytz
import time
import numpy as np
import pandas as pd
import pickle
from datetime import datetime, timedelta
from enum import Enum
from collections import namedtuple, defaultdict
import gzip
import random
import os
import json

from acnportal import acnsim
from acnportal import algorithms
from acnportal.acnsim.events import EventQueue
# from acnportal.signals.tariffs.tou_tariff import TimeOfUseTariff
from utility_functions.signals.tariffs.tou_tariff import TimeOfUseTariff
from acnportal.acnsim.events import GaussianMixtureEvents
from acnportal.contrib.acnsim import StochasticNetwork
from acnportal.acnsim.network import ChargingNetwork
from modified_adacharge import modified_adacharge
from modified_simulator import modified_simulator
# from acnportal.acnsim import analysis
from utility_functions import analysis
# from modified_evse import *
from acnportal.acnsim.models.evse import get_evse_by_type
# import sqlite3
# from sqlalchemy import create_engine

In [4]:
# This method won't take a evse_per_phase argument
def ev_fleet_level_2_network(transformer_cap=30):
    """ Configurable charging network for level-2 EVSEs connected line to line
        at 415 V.

    Args:
        transformer_cap (float): Capacity of the transformer feeding the network
          [kW]
        evse_per_phase (int): Number of EVSEs on each phase. Total number of
          EVSEs will be 3 * evse_per_phase.

    Returns:
        ChargingNetwork: Configured ChargingNetwork.
    """
    network = StochasticNetwork(early_departure=True)
    # network = ChargingNetwork()
    voltage = 415
    evse_type = 'AeroVironment'
    # evse_type = 'ClipperCreek'

    # Define the sets of EVSEs in the Caltech ACN.
    AB_ids = ['AB-{0}'.format(i) for i in range(3)]
    BC_ids = ['BC-{0}'.format(i) for i in range(3)]
    CA_ids = ['CA-{0}'.format(i) for i in range(3)]

    # print(AB_ids)

    # Add Caltech EVSEs
    for evse_id in AB_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 30)
    for evse_id in BC_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, -90)
    for evse_id in CA_ids:
        network.register_evse(get_evse_by_type(evse_id, evse_type), voltage, 150)

    # Add Caltech Constraint Set
    AB = acnsim.Current(AB_ids)
    BC = acnsim.Current(BC_ids)
    CA = acnsim.Current(CA_ids)

    # Define intermediate currents
    I3a = AB - CA
    I3b = BC - AB
    I3c = CA - BC
    I2a = (1 / 4) * (I3a - I3c)
    I2b = (1 / 4) * (I3b - I3a)
    I2c = (1 / 4) * (I3c - I3b)

    # Build constraint set
    primary_side_constr = transformer_cap * 1000 / 3 / 415
    print(primary_side_constr)
    secondary_side_constr = transformer_cap * 1000 / 3 / 230
    print(secondary_side_constr)
    network.add_constraint(I3a, secondary_side_constr, name='Secondary A')
    network.add_constraint(I3b, secondary_side_constr, name='Secondary B')
    network.add_constraint(I3c, secondary_side_constr, name='Secondary C')
    network.add_constraint(I2a, primary_side_constr, name='Primary A')
    network.add_constraint(I2b, primary_side_constr, name='Primary B')
    network.add_constraint(I2c, primary_side_constr, name='Primary C')

    return network

In [5]:
# How long each time discrete time interval in the simulation should be.
PERIOD = 5  # minutes

# Voltage of the network.
VOLTAGE = 415  # volts      

# Default maximum charging rate for each EV battery.
DEFAULT_BATTERY_POWER = 11 # kW

#### Generate synthetic events for simulation

In [6]:
# Generate synthetic events based on JPL data provided by ACN-Sim.
class CustomUnpicklerJPLdata(pickle.Unpickler):
    def find_class(self, module, name):
        if name == "sklearn.mixture.gaussian_mixture":
            return sklearn.mixture.GaussianMixture
        if name == "GaussianMixture":
            return sklearn.mixture.GaussianMixture
        return super().find_class(module, name)
    
def get_synth_events(sessions_per_day):
    gmm = CustomUnpicklerJPLdata(open('./data/jpl_weekday_40.pkl', "rb")).load()


    # Generate a list of the number of sessions to draw for each day.
    # This generates 30 days of charging demands.
    # num_evs = [0]*2 + [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2 + \
    #           [sessions_per_day]*5 + [0]*2 + [sessions_per_day]*5 + [0]*2

    # Generate sessions for 1 day (weekdays only)
    num_evs = [0]*2 + [sessions_per_day]*1

    # Note that because we are drawing from a distribution, some sessions will be
    # invalid, we ignore these sessions and remove the corresponding plugin events.
    # gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=0.08334)
    gen = GaussianMixtureEvents(pretrained_model=gmm, duration_min=5)

    synth_events = gen.generate_events(num_evs, PERIOD, VOLTAGE, DEFAULT_BATTERY_POWER)
    return synth_events

In [7]:
# 27 EV sessions per weekday
events_27_ev = get_synth_events(27)

c:\Users\s3955218\Anaconda3\envs\evsim\lib\site-packages\sklearn\base.py:288: UserWarning: Trying to unpickle estimator GaussianMixture from version 0.20.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
# Checking session details
session_copy = deepcopy(events_27_ev)
# print(type(session_copy))
session_list = list(session_copy.queue)
for session in session_list:
    print(session[0], session[1].ev._session_id, session[1].ev.arrival, \
          session[1].ev.departure, session[1].ev.requested_energy, session[1].event_type)

648 session_17 648 734 11.554954389992643 Plugin
648 session_19 648 753 7.770639251154674 Plugin
655 session_2 655 715 15.355959883343681 Plugin
650 session_13 650 754 13.425874251041241 Plugin
649 session_18 649 756 7.374409424613267 Plugin
656 session_23 656 716 26.411869199639273 Plugin
660 session_14 660 770 20.183826499676172 Plugin
674 session_3 674 734 9.395039564108854 Plugin
655 session_8 655 769 2.9063820674279865 Plugin
659 session_9 659 779 3.2429038515051363 Plugin
661 session_22 661 767 8.625865511567024 Plugin
668 session_11 668 777 7.3181679217013365 Plugin
657 session_25 657 721 24.277511946126385 Plugin
756 session_6 756 816 15.706458557243046 Plugin
665 session_1 665 754 8.3279699469011 Plugin
761 session_7 761 821 11.293886610027375 Plugin
758 session_16 758 818 9.374281481777098 Plugin
729 session_0 729 789 15.163394759830517 Plugin
667 session_15 667 784 18.969728990538282 Plugin
680 session_4 680 740 9.8353786067874 Plugin
671 session_20 671 760 11.70086399318039

#### Algorithms and new equation

In [23]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['RR'] = algorithms.RoundRobin(algorithms.first_come_first_served, continuous_inc=1)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_without_priority_ev),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_for_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=300, max_recompute=1)

#### Run Simulation with New Equation

In [36]:
sims = dict()
def run_experiment(alg_name, cap):
    """ Run simulation for the events defined previously and the specified
        network / algorithm / events.
    """
    # Timezone of the ACN we are using.
    # timezone = pytz.timezone('America/Los_Angeles')
    # Australian timezone for the experiment
    timezone = pytz.timezone('Australia/Melbourne')

    start = timezone.localize(datetime(2025, 5, 20))
    end = timezone.localize(datetime(2025, 5, 20))

    cn = ev_fleet_level_2_network(transformer_cap=cap)

    alg = deepcopy(sch[alg_name])
    alg.max_rate_estimator = algorithms.SimpleRampdown()
    alg.estimate_max_rate = True
    experiment_events = deepcopy(events_27_ev)
    signals = {'tariff': TimeOfUseTariff('sce_tou_ev_4_march_2019')}

    sim = modified_simulator.Simulator(cn, alg, experiment_events, start, period=PERIOD, signals=signals, verbose=False)
    print("Running...")
    start_simulation = time.time()
    if alg_name == "MPC_Offline":
        alg.register_events(experiment_events)
        alg.solve()
    # if alg_name == "MPC":
    #     sim.high_priority_ev_sessions = ["session_19", "session_7", "session_15", "session_10"]
    # sim.high_priority_ev_sessions = ["session_4", "session_12", "session_2"]
    sim.run()
    print(f"Run time: {time.time() - start_simulation}")

    return sim

In [37]:
warnings.simplefilter("ignore")

capacities = list(range(80, 110, 5))
alg_names = ["Unctrl", "RR", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            try:
                sims[config] = run_experiment(*config)
            except Exception as e:
                print(f"InfeasibilityException: {e}")
                sims[config] = None
                continue

('Unctrl', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 0.30292630195617676
('RR', 80)
64.2570281124498
115.94202898550725
InfeasibilityException: 'RR'
('MPC', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 14.520563125610352
('Unctrl', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 0.37746500968933105
('RR', 85)
68.27309236947791
123.18840579710144
InfeasibilityException: 'RR'
('MPC', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 13.865756273269653
('Unctrl', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 0.3475813865661621
('RR', 90)
72.28915662650603
130.43478260869566
InfeasibilityException: 'RR'
('MPC', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 12.168213129043579
('Unctrl', 95)
76.30522088353413
137.68115942028987
Running...
Run time: 0.3240530490875244
('RR', 95)
76.30522088353413
137.68115942028987
InfeasibilityException: 'RR'
('MPC', 95)
76.30522088353413
137.68115942028987
Running..

In [6]:
# result_dir = "results/sims/new_simulation"
# if not os.path.exists(result_dir):
#     os.makedirs(result_dir)
    
# for config, sim in sims.items():
#     name = "results/sims/new_simulation/{0}-{1}.json.gz".format(*config)
#     if not os.path.exists(name):
#         data = sim.to_json()
#         with gzip.GzipFile(name, 'w') as fout:
#             fout.write(json.dumps(data).encode('utf-8'))

#### Result Analysis

In [38]:
def calc_metrics(config, sim):
    metrics = {
        "Network Type": "three_phase",
        "Algorithm": config[0],
        "Capacity (kW)": config[1],
        "Energy Delivered (%)": analysis.proportion_of_energy_delivered(sim) * 100,
        "Demand met": analysis.proportion_of_demands_met(sim) * 100,
        "Max Utilization (%)": np.max(analysis.aggregate_power(sim)) / config[1] * 100,
        "energy_cost": analysis.energy_cost(sim),
        "total_energy_delivered": analysis.total_energy_delivered(sim),
        "Energy Delivered to priority evs (%)": analysis.proportion_of_priority_evs_energy_delivered(sim, ["session_23",
                                                                                                           "session_13",
                                                                                                           "session_6",
                                                                                                           "session_17",
                                                                                                           "session_14",
                                                                                                           "session_0",
                                                                                                           "session_16"
                                                                                                           ]),
        "Peak (kW)": np.max(analysis.aggregate_power(sim))
    }
    metrics["Current Unbalance"] = np.nanmean(analysis.current_unbalance(sim, ['Secondary {0}'.format(p) for p in 'ABC'], 'NEMA'))

    return metrics

In [39]:
# filter the sims dict to only include the ones that are not None
sims = {k: v for k, v in sims.items() if v is not None}

In [40]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [41]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Energy Delivered to priority evs (%),Peak (kW),Current Unbalance
0,three_phase,Unctrl,80,100.000,100.000,123.750,21.071,297.349,100.0,99.000,0.289
1,three_phase,MPC,80,80.218,40.741,86.131,18.647,238.527,100.0,68.905,0.414
2,three_phase,Unctrl,85,100.000,100.000,116.471,21.071,297.349,100.0,99.000,0.286
3,three_phase,MPC,85,79.803,44.444,77.647,18.539,237.295,100.0,66.000,0.416
4,three_phase,Unctrl,90,100.000,100.000,110.000,21.071,297.349,100.0,99.000,0.246
5,three_phase,MPC,90,80.561,44.444,72.639,18.735,239.546,100.0,65.375,0.391
6,three_phase,Unctrl,95,100.000,100.000,104.211,21.071,297.349,100.0,99.000,0.255
7,three_phase,MPC,95,80.311,44.444,69.474,18.671,238.804,100.0,66.000,0.394
8,three_phase,Unctrl,100,100.000,100.000,99.000,21.071,297.349,100.0,99.000,0.286
9,three_phase,MPC,100,80.200,40.741,66.000,18.642,238.475,100.0,66.000,0.401


#### Simulation with different algorithms

In [42]:
sch = {}
sch['Unctrl'] = algorithms.UncontrolledCharging()
sch['LLF'] = algorithms.SortedSchedulingAlgo(algorithms.least_laxity_first)

cost_min_obj = [
                modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty),
                modified_adacharge.ObjectiveComponent(modified_adacharge.tou_energy_cost_with_pv),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_for_priority_ev, 2),
                # modified_adacharge.ObjectiveComponent(modified_adacharge.non_completion_penalty_without_priority_ev),
                modified_adacharge.ObjectiveComponent(modified_adacharge.quick_charge, 1e-6)
               ]

sch['MPC'] = modified_adacharge.AdaptiveSchedulingAlgorithm(cost_min_obj, solver="MOSEK", quantize=True, reallocate=False, peak_limit=200, max_recompute=1)

In [43]:
sims = dict()

In [44]:
warnings.simplefilter("ignore")

capacities = list(range(80, 110, 5))
alg_names = ["Unctrl", "LLF", "MPC"]

for cap in capacities:
    for alg_name in alg_names:
        config = (alg_name, cap)
        print(config)
        if config not in sims:
            sims[config] = run_experiment(*config)

('Unctrl', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 0.3032107353210449
('LLF', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 0.8962094783782959
('MPC', 80)
64.2570281124498
115.94202898550725
Running...
Run time: 13.599734783172607
('Unctrl', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 0.3433492183685303
('LLF', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 0.8645901679992676
('MPC', 85)
68.27309236947791
123.18840579710144
Running...
Run time: 14.099763870239258
('Unctrl', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 0.3337514400482178
('LLF', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 0.8331298828125
('MPC', 90)
72.28915662650603
130.43478260869566
Running...
Run time: 13.716216564178467
('Unctrl', 95)
76.30522088353413
137.68115942028987
Running...
Run time: 0.3144092559814453
('LLF', 95)
76.30522088353413
137.68115942028987
Running...
Run time: 0.9856274127960205
('MPC', 95)
76.

In [45]:
# filter the sims dict to only include the ones that are not None
sims = {k: v for k, v in sims.items() if v is not None}

In [46]:
metrics = pd.DataFrame(calc_metrics(config, sim) for config, sim in sims.items()).round(3)
metrics.rename(columns={"Capacity (kW)": "capacity"}, inplace=True)

In [47]:
metrics

,Network Type,Algorithm,capacity,Energy Delivered (%),Demand met,Max Utilization (%),energy_cost,total_energy_delivered,Energy Delivered to priority evs (%),Peak (kW),Current Unbalance
0,three_phase,Unctrl,80,100.000,100.000,123.750,21.071,297.349,100.000,99.000,0.295
1,three_phase,LLF,80,88.103,81.481,87.944,19.678,261.974,99.841,70.355,0.319
2,three_phase,MPC,80,80.567,44.444,87.169,18.737,239.564,100.000,69.735,0.394
3,three_phase,Unctrl,85,100.000,100.000,116.471,21.071,297.349,100.000,99.000,0.261
4,three_phase,LLF,85,88.121,81.481,79.841,19.681,262.026,99.841,67.865,0.315
5,three_phase,MPC,85,79.850,40.741,76.418,18.552,237.434,100.000,64.955,0.411
6,three_phase,Unctrl,90,100.000,100.000,110.000,21.071,297.349,100.000,99.000,0.341
7,three_phase,LLF,90,88.068,81.481,78.172,19.671,261.870,99.717,70.355,0.308
8,three_phase,MPC,90,80.485,44.444,73.333,18.716,239.321,100.000,66.000,0.417
9,three_phase,Unctrl,95,100.000,100.000,104.211,21.071,297.349,100.000,99.000,0.305
